In [9]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 18: Settlers of The North Pole ---</h2><p>On the outskirts of the North Pole base construction project, many Elves are collecting <span title="Trade wood for sheep?">lumber</span>.</p>
<p>The lumber collection area is 50 acres by 50 acres; each acre can be either <em>open ground</em> (<code>.</code>), <em>trees</em> (<code>|</code>), or a <em>lumberyard</em> (<code>#</code>). You take a scan of the area (your puzzle input).</p>
<p>Strange magic is at work here: each minute, the landscape looks entirely different. In exactly <em>one minute</em>, an open acre can fill with trees, a wooded acre can be converted to a lumberyard, or a lumberyard can be cleared to open ground (the lumber having been sent to other projects).</p>
<p>The change to each acre is based entirely on <em>the contents of that acre</em> as well as <em>the number of open, wooded, or lumberyard acres adjacent to it</em> at the start of each minute. Here, "adjacent" means any of the eight acres surrounding that acre. (Acres on the edges of the lumber collection area might have fewer than eight adjacent acres; the missing acres aren't counted.)</p>
<p>In particular:</p>
<ul>
<li>An <em>open</em> acre will become filled with <em>trees</em> if <em>three or more</em> adjacent acres contained trees. Otherwise, nothing happens.</li>
<li>An acre filled with <em>trees</em> will become a <em>lumberyard</em> if <em>three or more</em> adjacent acres were lumberyards. Otherwise, nothing happens.</li>
<li>An acre containing a <em>lumberyard</em> will remain a <em>lumberyard</em> if it was adjacent to <em>at least one other lumberyard and at least one acre containing trees</em>. Otherwise, it becomes <em>open</em>.</li>
</ul>
<p>These changes happen across all acres <em>simultaneously</em>, each of them using the state of all acres at the beginning of the minute and changing to their new form by the end of that same minute. Changes that happen during the minute don't affect each other.</p>
<p>For example, suppose the lumber collection area is instead only 10 by 10 acres with this initial configuration:</p>
<pre><code>Initial state:
.#.#...|#.
.....#|##|
.|..|...#.
..|#.....#
#.#|||#|#|
...#.||...
.|....|...
||...#|.#|
|.||||..|.
...#.|..|.

After 1 minute:
.......##.
......|###
.|..|...#.
..|#||...#
..##||.|#|
...#||||..
||...|||..
|||||.||.|
||||||||||
....||..|.

After 2 minutes:
.......#..
......|#..
.|.|||....
..##|||..#
..###|||#|
...#|||||.
|||||||||.
||||||||||
||||||||||
.|||||||||

After 3 minutes:
.......#..
....|||#..
.|.||||...
..###|||.#
...##|||#|
.||##|||||
||||||||||
||||||||||
||||||||||
||||||||||

After 4 minutes:
.....|.#..
...||||#..
.|.#||||..
..###||||#
...###||#|
|||##|||||
||||||||||
||||||||||
||||||||||
||||||||||

After 5 minutes:
....|||#..
...||||#..
.|.##||||.
..####|||#
.|.###||#|
|||###||||
||||||||||
||||||||||
||||||||||
||||||||||

After 6 minutes:
...||||#..
...||||#..
.|.###|||.
..#.##|||#
|||#.##|#|
|||###||||
||||#|||||
||||||||||
||||||||||
||||||||||

After 7 minutes:
...||||#..
..||#|##..
.|.####||.
||#..##||#
||##.##|#|
|||####|||
|||###||||
||||||||||
||||||||||
||||||||||

After 8 minutes:
..||||##..
..|#####..
|||#####|.
||#...##|#
||##..###|
||##.###||
|||####|||
||||#|||||
||||||||||
||||||||||

After 9 minutes:
..||###...
.||#####..
||##...##.
||#....###
|##....##|
||##..###|
||######||
|||###||||
||||||||||
||||||||||

After 10 minutes:
.||##.....
||###.....
||##......
|##.....##
|##.....##
|##....##|
||##.####|
||#####|||
||||#|||||
||||||||||
</code></pre>

<p>After 10 minutes, there are <code>37</code> wooded acres and <code>31</code> lumberyards.  Multiplying the number of wooded acres by the number of lumberyards gives the total <em>resource value</em> after ten minutes: <code>37 * 31 = <em>1147</em></code>.</p>
<p><em>What will the total resource value of the lumber collection area be after 10 minutes?</em></p>
</article>


In [27]:
from copy import deepcopy


example = """
.#.#...|#.
.....#|##|
.|..|...#.
..|#.....#
#.#|||#|#|
...#.||...
.|....|...
||...#|.#|
|.||||..|.
...#.|..|.
"""


class Area:
    adjacent = ((-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1))

    def __init__(self, s: str) -> None:
        self.area = [list(l) for l in s.strip().splitlines()]
        self.rows, self.cols = len(self.area), len(self.area[0])

    def time_elapse(self, minutes: int, do_print: bool = False) -> Area:
        if do_print:
            print("Initial state:")
            print(self)
            print()

        old_area = deepcopy(self.area)

        for i in range(minutes):
            for r, c in product(range(self.rows), range(self.cols)):
                if old_area[r][c] == ".":
                    # An open acre will become filled with trees
                    if (
                        sum(
                            1
                            for dr, dc in self.adjacent
                            if 0 <= r + dr < self.rows
                            and 0 <= c + dc < self.cols
                            and old_area[r + dr][c + dc] == "|"
                        )
                        >= 3
                    ):
                        #   - if three or more adjacent acres contained trees.
                        self.area[r][c] = "|"
                        #   - Otherwise, nothing happens.
                elif old_area[r][c] == "|":
                    # An acre filled with trees will become a lumberyard
                    if (
                        sum(
                            1
                            for dr, dc in self.adjacent
                            if 0 <= r + dr < self.rows
                            and 0 <= c + dc < self.cols
                            and old_area[r + dr][c + dc] == "#"
                        )
                        >= 3
                    ):
                        #   - if three or more adjacent acres were lumberyards.
                        self.area[r][c] = "#"
                        #   - Otherwise, nothing happens.
                elif old_area[r][c] == "#":
                    # An acre containing a lumberyard will remain a lumberyard
                    if not (
                        sum(
                            1
                            for dr, dc in self.adjacent
                            if 0 <= r + dr < self.rows
                            and 0 <= c + dc < self.cols
                            and old_area[r + dr][c + dc] == "#"
                        )
                        >= 1
                        and sum(
                            1
                            for dr, dc in self.adjacent
                            if 0 <= r + dr < self.rows
                            and 0 <= c + dc < self.cols
                            and old_area[r + dr][c + dc] == "|"
                        )
                        >= 1
                    ):
                        #   - if it was adjacent to at least one other lumberyard and
                        #     at least one acre containing trees.
                        #   - Otherwise, it becomes open.
                        self.area[r][c] = "."

            old_area = deepcopy(self.area)

            if do_print:
                print(f"After {i + 1} minute{'s' if i > 0 else ''}:")
                print(self)
                print()

        return self

    def total_resource_value(self) -> int:
        return sum(1 for l in self.area for c in l if c == "|") * sum(
            1 for l in self.area for c in l if c == "#"
        )

    def __repr__(self) -> str:
        return "\n".join("".join(l) for l in self.area)


print(
    f"Example: {Area(example).time_elapse(10).total_resource_value()}, should be 37 * 31 = 1147"
)

Example: 1147, should be 37 * 31 = 1147


In [28]:
with open("../input/day18.txt") as f:
    puzzle = f.read()

print(f"Part I: {Area(puzzle).time_elapse(10).total_resource_value()}")

Part I: 531417


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>531417</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>This important natural resource will need to last for at least thousands of years.  Are the Elves collecting this lumber sustainably?</p>
<p><em>What will the total resource value of the lumber collection area be after 1000000000 minutes?</em></p>
</article>

</main>


In [32]:
print(f"PartI I: {Area(puzzle).time_elapse(1_000).total_resource_value()}")
# becomes stabel after +- 1000 minutes

PartI I: 205296


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>205296</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
